In [1]:
import pandas as pd
import numpy as np
import cvxpy as cp
from scipy import sparse

import time 

import matplotlib.pyplot as plt
%matplotlib inline

from multiprocessing import Process, Pool
from itertools import repeat

import matplotlib
matplotlib.rcParams['font.family'] = 'Times New Roman'

### This notebook identifies the parameters of the virtual batteries and power lines in the San Francisco Bay area
- Results for the base case 
- Plot the figures to illustrate their impacts

### Step 1: Select trip chains and obtain parameters

In [3]:
index_to_county = {
    0: 'San Francisco',
    1: 'San Mateo',
    2: 'Santa Clara',
    3: 'Alameda',
    4: 'Contra Costa',
    5: 'Solano',
    6: 'Napa',
    7: 'Sonoma',
    8: 'Marin'
}


# key代表node功率注入，value第一个数代表注入的line power，value第二个数代表流出的line power
line_to_county = {
    0: (0, 8),
    1: (1, 0),
    2: (2, 1),
    3: (3, 2),
    4: (4, 3),
    5: (5, 4),
    6: (6, 5),
    7: (7, 6),
    8: (8, 7)
}


In [4]:
N_t = 96
N_n = 9
N_line = 9 # number of power lines, each line starts from node 0 and ends at node k

L = np.zeros((N_t, N_t))
for i in range(N_t):
    for j in range(N_t):
        if i >= j:
            L[i, j] = 1

In [5]:
ell_profile_init = cp.Variable((N_n, N_t))
E_profile_init = cp.Variable((N_n, N_t))
ell_UB_init = cp.Parameter((N_n, N_t))
E_UB_init = cp.Parameter((N_n, N_t))
E_LB_init = cp.Parameter((N_n, N_t))

c_init = [
    ell_profile_init >= 0,
    ell_profile_init <= ell_UB_init,
    E_profile_init >= E_LB_init,
    E_profile_init <= E_UB_init,
]

for n in range(N_n):
    c_init += [
        L @ ell_profile_init[n, :] * 0.25 == E_profile_init[n, :],
    ]

obj_init_min = cp.Minimize(cp.sum(ell_profile_init))
prob_init_min = cp.Problem(obj_init_min, c_init)

obj_init_max = cp.Maximize(cp.sum(ell_profile_init))
prob_init_max = cp.Problem(obj_init_max, c_init)


def initialProfile(ell_UB_init_value, E_UB_init_value, E_LB_init_value):
    ell_UB_init.value = ell_UB_init_value
    E_UB_init.value = E_UB_init_value
    E_LB_init.value = E_LB_init_value
    prob_init_min.solve(
        solver=cp.MOSEK,
        # verbose = True,
        warm_start = True,
        # MIPGap = 0.01
        )
    ell_init_min = ell_profile_init.value
    prob_init_max.solve(
        solver=cp.MOSEK,
        # verbose = True,
        warm_start = True,
        # MIPGap = 0.01
        )
    ell_init_max = ell_profile_init.value
    ell_nominal_matrix = (ell_init_min + ell_init_max) / 2
    # ell_nominal_matrix = ell_init_min

    E_nominal_matrix = np.zeros((N_n, N_t))
    for n in range(N_n):
        E_nominal_matrix[n] = L @ ell_nominal_matrix[n] * 0.25

    return ell_nominal_matrix, E_nominal_matrix

In [6]:
def VirtualBattey(ell_nominal_matrix, E_nominal_matrix, ell_UB_value, E_UB_value, E_LB_value):
    u_UB = np.zeros((N_n, N_t))
    u_LB = np.zeros((N_n, N_t))
    E_UB_battery = np.zeros((N_n, N_t))
    E_LB_battery = np.zeros((N_n, N_t))

    for n in range(N_n):
        u_UB[n] = (ell_UB_value[n] - ell_nominal_matrix[n])
        u_LB[n] = (0 - ell_nominal_matrix[n])
        E_UB_battery[n] = (E_UB_value[n] - E_nominal_matrix[n])
        E_LB_battery[n] = (E_LB_value[n] - E_nominal_matrix[n])

    return u_UB, u_LB, E_UB_battery, E_LB_battery
    

In [7]:
f_UB = cp.Variable((N_line, N_t))
f_LB = cp.Variable((N_line, N_t))

ell_UB_county = cp.Parameter((N_n, N_t))
E_UB_county = cp.Parameter((N_n, N_t))
E_LB_county = cp.Parameter((N_n, N_t))

ell_nominal = cp.Parameter((N_n, N_t))
E_nominal = cp.Parameter((N_n, N_t))


""" Constraints from definitions """
c_line = [
    f_UB >= f_LB,
    f_LB <= 0,
    f_UB >= 0,
    # f_UB - f_LB >= 0.01,
]

"""Constraints for each county"""

for n in range(N_n):
    inj_node, out_node = line_to_county[n]
    c_line += [
        f_UB[inj_node] - f_LB[out_node] + ell_nominal[n] <= ell_UB_county[n],
        f_LB[inj_node] - f_UB[out_node] + ell_nominal[n] >= 0,
        0.25 * L @ (f_UB[inj_node] - f_LB[out_node] + ell_nominal[n]) <= E_UB_county[n],
        0.25 * L @ (f_LB[inj_node] - f_UB[out_node] + ell_nominal[n]) >= E_LB_county[n],
    ]


obj_line = cp.Maximize(
    cp.sum([cp.sum(f_UB[k]-f_LB[k]) - 0.1*cp.norm(f_UB[k]-cp.mean(f_UB[k]), 2) 
            - 0.1*cp.norm(f_LB[k]-cp.mean(f_LB[k]), 2) for k in range(N_line)])
    )

# obj_line = cp.Maximize(
#     cp.sum([cp.sum(f_UB[k]-f_LB[k]) for k in range(N_line)])
#     )

prob_line = cp.Problem(obj_line, c_line)

def VirtualLine(ell_nominal_matrix, E_nominal_matrix, ell_UB_value, E_UB_value, E_LB_value):
    ell_nominal.value = ell_nominal_matrix
    E_nominal.value = E_nominal_matrix
    ell_UB_county.value = ell_UB_value
    E_UB_county.value = E_UB_value
    E_LB_county.value = E_LB_value

    prob_line.solve(
        solver=cp.MOSEK,
        # verbose = True,
        warm_start = True,
        # MIPGap = 0.01
        )

    f_UB_value = f_UB.value
    f_LB_value = f_LB.value

    return f_UB_value, f_LB_value

In [11]:
capacityNum = 1
capacity_scale = [1.0]
peneList = [0.025]
peneNum = len(peneList)

ratio_pubCharger_list = [0.4]
ratioNum = len(ratio_pubCharger_list)

In [12]:
u_UB_rec = {}
u_LB_rec = {}
E_UB_rec = {}
E_LB_rec = {}
f_UB_rec = {}
f_LB_rec = {}

for i in range(peneNum):
    u_UB_rec[i] = {}
    u_LB_rec[i] = {}
    E_UB_rec[i] = {}
    E_LB_rec[i] = {}
    f_UB_rec[i] = {}
    f_LB_rec[i] = {}
    pene_value = peneList[i]
    path_temp = f'Results/Bay/AggregateModel_realPene/pene={pene_value:.3f}'
    for ratio_pubCharger in ratio_pubCharger_list:
        u_UB_rec[i][ratio_pubCharger] = {}
        u_LB_rec[i][ratio_pubCharger] = {}
        E_UB_rec[i][ratio_pubCharger] = {}
        E_LB_rec[i][ratio_pubCharger] = {}
        f_UB_rec[i][ratio_pubCharger] = {}
        f_LB_rec[i][ratio_pubCharger] = {}
        for j in range(capacityNum):
            ell_UB_temp = sparse.load_npz(f"{path_temp}/ell_UB_new_ratio={ratio_pubCharger:.1f}_scale={capacity_scale[j]}.npz").toarray()[0]
            E_UB_temp = sparse.load_npz(f"{path_temp}/E_UB_ratio={ratio_pubCharger:.1f}_scale={capacity_scale[j]}.npz").toarray()[0]
            E_LB_temp = sparse.load_npz(f"{path_temp}/E_LB_ratio={ratio_pubCharger:.1f}_scale={capacity_scale[j]}.npz").toarray()[0]
            ell_UB_temp = ell_UB_temp.reshape(N_n, N_t)
            E_UB_temp = E_UB_temp.reshape(N_n, N_t)
            E_LB_temp = E_LB_temp.reshape(N_n, N_t)

            """get the initial charging power profile"""
            ell_nominal_temp, E_nominal_temp = initialProfile(ell_UB_temp, E_UB_temp, E_LB_temp)

            """get the parameters of virtual batteries"""
            u_UB_temp, u_LB_temp, E_UB_battery_temp, E_LB_battery_temp = VirtualBattey(
                ell_nominal_temp, E_nominal_temp, ell_UB_temp, E_UB_temp, E_LB_temp)
            u_UB_rec[i][ratio_pubCharger][j] = u_UB_temp
            u_LB_rec[i][ratio_pubCharger][j] = u_LB_temp
            E_UB_rec[i][ratio_pubCharger][j] = E_UB_battery_temp
            E_LB_rec[i][ratio_pubCharger][j] = E_LB_battery_temp


            """get the parameters of virtual lines"""
            f_UB_temp, f_LB_temp = VirtualLine(ell_nominal_temp, E_nominal_temp, ell_UB_temp, E_UB_temp, E_LB_temp)
            f_UB_rec[i][ratio_pubCharger][j] = f_UB_temp
            f_LB_rec[i][ratio_pubCharger][j] = f_LB_temp

            break # only one capacity scale
        break # only one ratio_pubCharger
    break # only one pene_value

            
            

In [93]:
import os
filePath = 'Results/Bay/VB_VL_realPene'
   
# for sessionNum in sessionNumList:
for i in [0]:
    for ratio_pubCharger in ratio_pubCharger_list:
        pene_value = float(peneList[i])
        path_temp = filePath + f'/pene={pene_value:.3f}_new'
        isExists=os.path.exists(path_temp) #判断路径是否存在，存在则返回true
        if not isExists:
            os.makedirs(path_temp)
           
        for j in range(capacityNum):

            """save the charging power profiles"""
            u_UB_sparse = sparse.csr_matrix(u_UB_rec[i][ratio_pubCharger][j])
            u_LB_sparse = sparse.csr_matrix(u_LB_rec[i][ratio_pubCharger][j])
            sparse.save_npz(f"{path_temp}/u_UB_ratio={ratio_pubCharger:.1f}_scale={capacity_scale[j]}.npz", u_UB_sparse)
            sparse.save_npz(f"{path_temp}/u_LB_ratio={ratio_pubCharger:.1f}_scale={capacity_scale[j]}.npz", u_LB_sparse)
            """save the energy bounds"""
            E_UB_sparse = sparse.csr_matrix(E_UB_rec[i][ratio_pubCharger][j])
            E_LB_sparse = sparse.csr_matrix(E_LB_rec[i][ratio_pubCharger][j])
            sparse.save_npz(f"{path_temp}/E_UB_ratio={ratio_pubCharger:.1f}_scale={capacity_scale[j]}.npz", E_UB_sparse)
            sparse.save_npz(f"{path_temp}/E_LB_ratio={ratio_pubCharger:.1f}_scale={capacity_scale[j]}.npz", E_LB_sparse)

            """save the power flow profiles"""
            f_UB_sparse = sparse.csr_matrix(f_UB_rec[i][ratio_pubCharger][j])
            f_LB_sparse = sparse.csr_matrix(f_LB_rec[i][ratio_pubCharger][j])
            sparse.save_npz(f"{path_temp}/f_UB_ratio={ratio_pubCharger:.1f}_scale={capacity_scale[j]}.npz", f_UB_sparse)
            sparse.save_npz(f"{path_temp}/f_LB_ratio={ratio_pubCharger:.1f}_scale={capacity_scale[j]}.npz", f_LB_sparse)
            print (f"ratio={ratio_pubCharger:.1f}, scale={capacity_scale[j]}, pene={peneList[i]}, saved")
            break # only one capacity scale
        break # only one ratio_pubCharger
    break # only one pene_value


ratio=0.4, scale=1.0, pene=0.025, saved
